# Lesson 9: Memory for Agents

This lesson explores the concept of adding **long-term memory** to agents, so they can persist and retrieve information over time. 

We’ll implement semantic, episodic, and procedural memory using the open-source mem0 library with Google's Gemini text embedding model, and a vector store that runs locally in the notebook, using ChromaDB. 


Learning Objectives:

1. Understand the different types of memory 
2. How to implement them, using the mem0 library.

## 1. Setup

First, we define some standard Magic Python commands to autoreload Python packages whenever they change:

In [1]:
%load_ext autoreload
%autoreload 2

### Set Up Python Environment

To set up your Python virtual environment using `uv` and use it in the Notebook, follow the step-by-step instructions from the [Course Admin](https://academy.towardsai.net/courses/take/agent-engineering/multimedia/67469688-lesson-1-part-2-course-admin) lesson from the beginning of the course.

**TL/DR:** Be sure the correct kernel pointing to your `uv` virtual environment is selected.

### Configure Gemini API

To configure the Gemini API, follow the step-by-step instructions from the [Course Admin](https://academy.towardsai.net/courses/take/agent-engineering/multimedia/67469688-lesson-1-part-2-course-admin) lesson.

But here is a quick check on what you need to run this Notebook:

1.  Get your key from [Google AI Studio](https://aistudio.google.com/app/apikey).
2.  From the root of your project, run: `cp .env.example .env` 
3.  Within the `.env` file, fill in the `GOOGLE_API_KEY` variable:

Now, the code below will load the key from the `.env` file:

In [ ]:
from lessons.utils import env

env.load(required_env_vars=["GOOGLE_API_KEY"])

Trying to load environment variables from `/Users/omar/Documents/ai_repos/course-ai-agents/.env`
Environment variables loaded successfully.


### Import Key Packages

In [ ]:
import os
import re
from typing import Optional

from google import genai
from mem0 import Memory

### Initialize the Gemini Client

In [ ]:
client = genai.Client()

### Define Constants

We will use the `gemini-2.5-flash` model, which is fast and cost-effective:

In [ ]:
MODEL_ID = "gemini-2.5-flash"

### Configure mem0 (Gemini LLM + embeddings + local vector store)

Here we instantiate mem0 with:

- LLM: our existing Gemini model (`MODEL_ID = "gemini-2.5-flash"`) for the summarization/extraction of facts.
- Embeddings: Gemini’s `text-embedding-004` (dimension 768).
- Vector store:
    - ChromaDB with `MEM_BACKEND=chromadb` 

In [ ]:
MEM0_CONFIG = {
    # Use Google's text-embedding-004 (768-dim) for embeddings
    "embedder": {
        "provider": "gemini",
        "config": {
            "model": "text-embedding-004",
            "embedding_dims": 768,
            "api_key": os.getenv("GOOGLE_API_KEY"),
        },
    },
    # Use ChromaDB as a local, in-notebook vector store (ephemeral, in-memory)
    "vector_store": {
        "provider": "chroma",
        "config": {
            "collection_name": "lesson9_memories",
        },
    },
    "llm": {
        "provider": "gemini",
        "config": {
            "model": MODEL_ID,
            "api_key": os.getenv("GOOGLE_API_KEY"),
        },
    },
}

memory = Memory.from_config(MEM0_CONFIG)
MEM_USER_ID = "lesson9_notebook_student"
memory.delete_all(user_id=MEM_USER_ID)
print("✅ Mem0 ready (Gemini embeddings + in-memory Chroma).")

✅ Mem0 ready (Gemini embeddings + in-memory Chroma).


### Helper functions: add/search memory

A small wrapper layer around mem0 to:

- **Save** a memory string (or a short conversation) and tag it with a `category` (`semantic`, `episodic`, `procedure`) plus metadata.
    - `mem_add_text` will store the raw string, with `infer=False` (no LLM used)
    - `mem_add_conversation` will store an LLM-generated summary (gemini-2.5-flash in our case) of the conversation with `infer=True`

- **Search** memory and return results for display. It can also filter by category.

For the metadata, mem0 only allows primitive types (str, int, float, bool, None) that is why we convert to `str` any non-primitive values.

In [ ]:
def mem_add_text(text: str, category: str = "semantic", **meta) -> str:
    """Add a single text memory. No LLM is used for extraction or summarization."""
    metadata = {"category": category}
    for k, v in meta.items():
        if isinstance(v, (str, int, float, bool)) or v is None:
            metadata[k] = v
        else:
            metadata[k] = str(v)
    memory.add(text, user_id=MEM_USER_ID, metadata=metadata, infer=False)
    return f"Saved {category} memory."


def mem_add_conversation(messages: list[dict], category: str = "episodic", **meta) -> str:
    """Add a conversation (list of {role, content}) as one episode."""
    metadata = {"category": category}
    for k, v in meta.items():
        metadata[k] = v if isinstance(v, (str, int, float, bool)) or v is None else str(v)
    memory.add(messages, user_id=MEM_USER_ID, metadata=metadata, infer=True)
    return f"Saved {category} episode."


def mem_search(query: str, limit: int = 5, category: Optional[str] = None) -> list[dict]:
    """
    Category-aware search wrapper.
    Returns the full result dicts so we can inspect metadata.
    """
    res = memory.search(query, user_id=MEM_USER_ID, limit=limit) or {}
    items = res.get("results", [])
    if category is not None:
        items = [r for r in items if (r.get("metadata") or {}).get("category") == category]
    return items

## 2. Semantic memory example (facts as atomic strings)

**Goal**: We show semantic memory as “facts & preferences” stored as short, individual strings.

- We insert a few example facts (e.g., “User has a dog named George”).

- Then we search with a natural query (e.g., “brother job”) and see the relevant fact returned.

In [ ]:
facts: list[str] = [
    "User prefers vegetarian meals.",
    "User has a dog named George.",
    "User is allergic to gluten.",
    "User's brother is named Mark and is a software engineer.",
]
for f in facts:
    print(mem_add_text(f, category="semantic"))

print("\nSearch --> 'brother job':")
print("\n".join(f"- {m}" for m in mem_search("brother job", limit=3)))

Saved semantic memory.
Saved semantic memory.
Saved semantic memory.
Saved semantic memory.

Search --> 'brother job':
- {'id': 'c6f0edbc-e0cc-40a2-abb4-d3f364b009ba', 'memory': "User's brother is named Mark and is a software engineer.", 'hash': '9a01dbd8ea8b96f8ed9c84e9dcdb55a1', 'metadata': {'category': 'semantic'}, 'score': 0.9269160032272339, 'created_at': '2025-08-25T17:26:22.183416-07:00', 'updated_at': None, 'user_id': 'lesson9_notebook_student', 'role': 'user'}
- {'id': 'bd98bf8d-8d85-4103-87b8-d6b32b8b9eb9', 'memory': 'User has a dog named George.', 'hash': '8c592a46b362bab2fd20a1a2a9214d74', 'metadata': {'category': 'semantic'}, 'score': 1.4484589099884033, 'created_at': '2025-08-25T17:26:21.261837-07:00', 'updated_at': None, 'user_id': 'lesson9_notebook_student', 'role': 'user'}
- {'id': 'ce5ccb85-60d2-4a67-8361-be990f9100f2', 'memory': 'User prefers vegetarian meals.', 'hash': '0034073d2dbe31e303972a0599565525', 'metadata': {'category': 'semantic'}, 'score': 1.5473814010620

## 3. Episodic memory example (summarize 3–4 turns → one episode)

**Goal**: Demonstrate episodic memory (experiences & history).

- We create a short 3–4 turn exchange between user and assistant.

- We ask the LLM to produce a concise episode summary (1–2 sentences) and save it under category="episodic".

- Finally, we run a semantic search (e.g., “deadline stress”) to retrieve that episode, we print the memory along with its creation timestamp.

This example show how an agent can compress transient chat into a single durable “moment.”

Since mem0 by default creates a created_at timestamp, we have the possibility to use it to sort and filter memories.
It would then be possible to answer questions like "What did we talk about last week?"

In [ ]:
# A short 4-turn exchange we want to compress into one "episode"
dialogue = [
    {"role": "user", "content": "I'm stressed about my project deadline on Friday."},
    {"role": "assistant", "content": "I’m here to help—what’s the blocker?"},
    {"role": "user", "content": "Mainly testing. I also prefer working at night."},
    {"role": "assistant", "content": "Okay, we can split testing into two sessions."},
]

# Ask the LLM to write a clear episodic summary.
episodic_prompt = f"""Summarize the following 3–4 turns as one concise 'episode' (1–2 sentences).
Keep salient details and tone.

{dialogue}
"""
summary_resp = client.models.generate_content(model=MODEL_ID, contents=episodic_prompt)
episode = summary_resp.text.strip()

print(
    mem_add_text(
        episode,
        category="episodic",
        summarized=True,
        turns=4,
    )
)

print("\nSearch --> 'deadline stress'")
hits = mem_search("deadline stress", limit=3, category="episodic")
for h in hits:
    print(f"- [created_at={h.get('created_at')}] {h['memory']}")

Saved episodic memory.

Search --> 'deadline stress'
- [created_at=2025-08-25T17:26:28.260452-07:00] Stressed about a looming Friday project deadline, the user identified testing as their main blocker and noted a preference for working at night. The assistant offered support by proposing they split the testing into two sessions.


## 4. Procedural memory example (learn & “run” a skill)

**Goal**: Demonstrate procedural memory (skills & workflows).

- We teach the agent a small procedure (e.g., monthly_report) by saving ordered steps in a single text block under category="procedure".

- We retrieve the procedure and parse the numbered steps to simulate “running” it.

This example shows how agents can learn reusable playbooks and trigger them later by name.

In [10]:
def learn_procedure(name: str, steps: list[str]) -> str:
    body = "Procedure: " + name + "\nSteps:\n" + "\n".join(f"{i + 1}. {s}" for i, s in enumerate(steps))
    return mem_add_text(body, category="procedure", procedure_name=name)


def find_procedure(name: str) -> dict | None:
    # search broadly but only keep category=procedure
    hits = mem_search(name, limit=10, category="procedure")
    # Prefer an exact name match if available
    for h in hits:
        if (h.get("metadata") or {}).get("procedure_name") == name:
            return h
    return hits[0] if hits else None


def run_procedure(name: str) -> str:
    p = find_procedure(name)
    if not p:
        return f"Procedure '{name}' not found."
    text = p.get("memory", "")
    steps = [m.group(1).strip() for m in re.finditer(r"^\s*\d+\.\s+(.*)$", text, flags=re.MULTILINE)]
    if not steps:
        return f"Procedure '{name}' has no parseable steps.\n\n{text}"
    lines = [f"→ {s}" for s in steps]
    return f"Running procedure '{name}':\n" + "\n".join(lines)


# Teach the agent a tiny recurrent skill:
print(
    learn_procedure(
        "monthly_report",
        [
            "Query sales DB for the last 30 days.",
            "Summarize top 5 insights.",
            "Ask user whether to email or display.",
        ],
    )
)

print("\nRetrieve and 'run' it:")
proc = find_procedure("monthly_report")
print(proc or "Not found.")

Saved procedure memory.

Retrieve and 'run' it:
{'id': 'ab0940fe-c9a4-446f-b3ff-cbaea099faee', 'memory': 'Procedure: monthly_report\nSteps:\n1. Query sales DB for the last 30 days.\n2. Summarize top 5 insights.\n3. Ask user whether to email or display.', 'hash': 'e66b82a0dcc57034cfa0a54084a643b5', 'metadata': {'procedure_name': 'monthly_report', 'category': 'procedure'}, 'score': 0.9708564281463623, 'created_at': '2025-08-25T17:26:28.965650-07:00', 'updated_at': None, 'user_id': 'lesson9_notebook_student', 'role': 'user'}
